# A simple SEA model of two rooms with a dividing wall

In this notebook we create a simple SEA model of two rooms divided by a concrete wall.

We start by importing some of the modules that are needed.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:.2e}'.format)
import matplotlib
%matplotlib inline

## Creating a SEA model

To create a SEA model we begin by creating an instance of `System`.

In [2]:
from seapy import System
from seapy.materials.materialsolid import modulus
from acoustics.signal import OctaveBand
f = OctaveBand(fstart=20.0, fstop=4000.0, fraction=1)

In [3]:
system1 = System(f)

We are only interested in a limited frequency range, e.g. the third octave bands ranging from 1000 to 4000 Hz.

### Materials

The rooms are filled with air, so we add air as material.

In [4]:
air = system1.add_material(
    'air', 
    'MaterialGas',
    density = 1.296,
    temperature = 293.0,
    bulk = 1.01e5,
    loss_factor=0.05,
    pressure=101325,
)

In [5]:
young=3.0e10
poisson=0.15

concrete = system1.add_material(
    'concrete', 
    'MaterialSolid', 
    young=young,
    poisson=poisson,
    density=2.3e3,
    loss_factor=0.02,
    temperature=293.,
    bulk=modulus("bulk", young=young, poisson=poisson),
    shear=modulus("bulk", young=young, poisson=poisson),
)

We don't know the shear modulus of concrete, so let's calculate it. With the function ``modulus`` we can calculate for an isotropic material any elastic modulus given two other ones.

In [6]:
from seapy.materials.materialsolid import modulus
concrete.shear = modulus('shear', young=3.0e10, poisson=0.15)

Just to be sure, we can list the properties of the concrete.

In [7]:
concrete.info([
    'density',
    'poisson',
    'young',
    'shear',
])

,15,31,63,125,251,501,1000,1995,3981
density,2.30e+03,2.30e+03,2.30e+03,2.30e+03,2.30e+03,2.30e+03,2.30e+03,2.30e+03,2.30e+03
poisson,1.50e-01,1.50e-01,1.50e-01,1.50e-01,1.50e-01,1.50e-01,1.50e-01,1.50e-01,1.50e-01
young,3.00e+10,3.00e+10,3.00e+10,3.00e+10,3.00e+10,3.00e+10,3.00e+10,3.00e+10,3.00e+10
shear,1.30e+10,1.30e+10,1.30e+10,1.30e+10,1.30e+10,1.30e+10,1.30e+10,1.30e+10,1.30e+10


### Rooms and wall

Now we add the two rooms.

In [8]:
room1 = system1.add_component(
    'room1', 
    'Component3DAcoustical', 
    material='air',
    length=4.0,
    height=2.5,
    width=5.0
)

TypeError: The object named room1 was given the following invalid keyword arguments: {'length', 'width', 'height'}

In [ ]:
room2 = system1.add_component('room2', 
                              'Component3DAcoustical', 
                              material='air',
                              length=5.0,
                              height=2.5,
                              width=5.0)

Given the material type and the volume we can for example calculate the mass of the air in the room

In [ ]:
room1.mass

or plot the modal density of the subsystem representing longitudinal waves

In [ ]:
fig = room1.subsystem_long.plot("modal_density", yscale='log')

We now add the concrete wall.

In [ ]:
wall = system1.add_component('wall', 
                             'Component2DPlate', 
                             material='concrete', 
                             length=3.0,
                             width=2.5,
                             height=0.05)

Let's have a look at the modal densities of the subsystems.

In [ ]:
system1.info(system1.subsystems, 'modal_density')

The modal density of the subsystem representing bending waves in the wall seems to remain constant.

It's also possible to inspect objects further. For example, as was shown with the mass of the room, it is possible to request e.g. multiple parameters.

In [ ]:
wall.subsystem_bend.info(['soundspeed_group', 
                          'soundspeed_phase', 
                          'modal_density',
                          'average_frequency_spacing',
                          'power_input',
                          'dlf',
                          'tlf',])

Shown is now a table, but what is returned is in fact a [pandas DataFrame](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.html). Pandas is a data analysis toolkit and offers powerful tools to analyse data and to export data to e.g. spreadsheet formats like Excel.

### Junction

The rooms and the wall form a junction and connect along a surface.

In [ ]:
junction1 = system1.add_junction('junction1', 'Junction', shape='Surface', components=['room1', 
                                                                                      'room2', 
                                                                                      'wall'])

Now, when we call ``junction1.update_couplings`` it tries to determine all the couplings between the subsystems of the components that were added.

In [ ]:
junction1.update_couplings()

We can now for example see the coupling loss factors of all the couplings that were added.

In [ ]:
system1.info(system1.couplings, 'clf')

Now that both the coupling loss factors and damping loss factors are known we can also list the total loss factor

In [ ]:
system1.info(system1.subsystems, 'tlf')

The coupling loss factor of the coupling between the rooms is based on the non-resonant transmission coefficient.

In [ ]:
system1.get_object('room1_SubsystemLong_room2_SubsystemLong').info(['tau', 'sound_reduction_index'])

In [ ]:
system1.get_object('wall_SubsystemBend_room1_SubsystemLong').info(['critical_frequency'])

### Excitation

We have defined the subsystems and couplings. What's left is to add an excitation to the system.

In [ ]:
excitation1 = room1.subsystem_long.add_excitation('excitation1', 
                                                 'ExcitationPointVolume', 
                                                 velocity=0.001,
                                                 radius=0.05)

The input power $P$ depends on the volume velocity $U$ of the source and the real part of the radiation impedance, i.e. the radiation resistance $R$.

In [ ]:
excitation1.info(['resistance'])

The resistance increases with frequency and therefore the radiated power increases similary.

In [ ]:
fig = excitation1.plot('power_level')

## Solving the system

Now we can solve the system.

In [ ]:
system1.solve()

We can have a look at the modal energy

In [ ]:
system1.info(system1.subsystems, 'modal_energy')

but those values are generally hard to interpret. Instead, we could just request the sound pressure levels in the rooms

In [ ]:
system1.info(['room1', 'room2'], 'pressure_level')

or plot them.

In [ ]:
fig = system1.plot(['room1', 'room2'], 'pressure_level')

Let's consider the sound pressure level difference between the two rooms.

In [ ]:
(room1.info(['pressure_level']) - room2.info(['pressure_level']))

In [ ]:
fig = system1.get_object('room1_SubsystemLong_room2_SubsystemLong').plot('sound_reduction_index')

Obviously, we can also look at the modal energies

In [ ]:
system1.info(system1.subsystems, 'modal_energy')

or see the level contributions of the individual subsystems.

In [ ]:
system1.info(system1.subsystems, 'velocity_level')

In [ ]:
system1.info(system1.subsystems, 'pressure_level')

## Path analysis and graphs

All the objects in `SeaPy` remember to which other objects they're connected. For example, we can list the subsystems in a component.

In [ ]:
for obj in wall.linked_subsystems:
    print(obj.name)

As soon as a model gets a bit bigger it can be hard to track which objects are connected. One way to help with keeping an overview is by drawing graphs.

In [ ]:
import networkx as nx

The following graph shows the relation between components and subsystems.

In [ ]:
G = system1.path_analysis.graph(['components', 'subsystems'])
nx.draw_networkx(G)

We can also show for example subsystems and couplings.

In [ ]:
G = system1.path_analysis.graph(['subsystems', 'couplings'])
fig = nx.draw_networkx(G)

In [ ]:
from seapy.tools import graph_couplings
G = graph_couplings(system1)
fig = nx.draw_networkx(G)

### Path analysis

By creating graphs of subsystems and couplings it is also straightforward to check whether subsystems are in anyway connected

In [ ]:
system1.path_analysis.has_path('room1_SubsystemLong', 'room2_SubsystemLong')

and to determine the possible paths between any two subsystems.

In [ ]:
for path in system1.path_analysis.paths('room1_SubsystemLong', 'room2_SubsystemLong'):
    print(path)

We can also calculate the level difference due to a transmission path.

In [ ]:
for path in system1.path_analysis.paths('room1_SubsystemLong', 'room2_SubsystemLong'):
    print(path.level_difference)

In [ ]:
list(system1.path_analysis.paths('room1_SubsystemLong', 'room2_SubsystemLong'))[0].level_difference

## Saving and restoring a model

SEA models can be saved as YAML.

In [ ]:
system1.save("model.yaml")

YAML is a human-readable file format. Models can be implemented or edited in the YAML file if desired.

In [ ]:
!head -n 20 model.yaml

Loading is done using the `load` method.

In [ ]:
system2 = System.load("model.yaml")

To verify whether the models are similar we check the modal energy.

In [ ]:
system1.info(system1.subsystems, 'modal_energy')

In [ ]:
system2.info(system2.subsystems, 'modal_energy')

That looks correctly. To be really sure we just calculate the modal energies again in the second model, to verify that other parameters have also been restored.

In [ ]:
system2.solve()
system2.info(system2.subsystems, 'modal_energy')

Same results.

## Source code and documentation

Source code of seapy can be found at [GitHub](https://github.com/FRidh/seapy) and documentation right [here](http://www.fridh.nl/seapy/).

In [ ]:
from IPython.display import IFrame
IFrame("https://seapy.readthedocs.io/en/latest/", width=800, height=600)